# Tool Result Compression

AI agents that interact with external tools and APIs often receive verbose responses containing extensive data structures, detailed metadata, debug information and fields irrelevant to the current task. When an agent calls a database query tool, API endpoint or file system operation, the raw result might include hundreds or thousands of tokens of JSON data, timestamps, internal identifiers, and system information that the agent does not need for decision-making. Including these complete tool results in the context window wastes valuable tokens and can overwhelm the agent with unnecessary detail, reducing its ability to focus on task-relevant information.

Tool result compression addresses this challenge by intelligently reducing the size of tool outputs before they enter the agent's context. Rather than passing raw API responses or complete data structures directly to the agent, the system applies selective field extraction, summarization, schema-based filtering, and intelligent compression that preserves only the information needed for the agent's current task. This maintains the agent's awareness of tool execution results while dramatically reducing token consumption, allowing more tools to be called and more context to be preserved within the same budget.

This notebook demonstrates how to implement tool result compression from basic field filtering to production-ready systems. We will explore selective field extraction from JSON responses, LLM-based summarization of verbose outputs, schema-driven compression with Pydantic models, structured truncation strategies, and complete tool result managers that automatically compress outputs based on configurable rules. These techniques are essential for building agents that use multiple tools extensively while maintaining efficient context management.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage
from pydantic import BaseModel, Field
from typing import List, Optional, Dict, Any, Union
from datetime import datetime
import json
import os

### Initialize language model for summarization

In [2]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY", "").strip(), temperature=0)

## The verbosity problem

Before implementing compression, we need to understand the scale of the verbosity problem with typical tool results. Many APIs and tools return rich data structures with extensive metadata, nested objects, arrays of related information, and system details that are valuable for debugging but unnecessary for agent decision-making. A simple database query might return row data along with query execution time, database version, connection details and internal identifiers. An API call might include response headers, rate limit information, pagination metadata, and nested related objects.

We will demonstrate this problem by simulating realistic tool results from common agent tools like database queries, API calls and file operations. This baseline shows the token cost of raw tool results and motivates the need for intelligent compression strategies.

To understand the verbosity problem, we will first create a simulation function that generates a realistic database query result. This represents what an agent might receive when querying a customer database - the result includes not just the customer data, but also extensive metadata about the query execution, connection details, and nested objects for addresses and preferences that may or may not be relevant to the agent's current task.

In [3]:
def simulate_database_query_result():
    """
    Simulate a database query result with extensive metadata.
    This represents a typical response from SQL databases, ORMs, and data access layers.
    
    Returns:
        dict: Verbose database query result with metadata and nested structures
    """
    return {
        # The original SQL query that was executed
        "query": "SELECT * FROM customers WHERE status = 'active' AND total_purchases > 1000",
        
        # Execution metadata - useful for debugging but rarely needed by agents
        "execution_time_ms": 45,
        "rows_examined": 15234,  # Total rows scanned
        "rows_returned": 3,      # Actual results returned
        
        # System metadata - almost never needed for agent decision-making
        "database_version": "PostgreSQL 14.5",
        "connection_id": "conn_9a7b3c2d",
        "query_id": "qry_2024112201",
        
        # The actual data results - this is what agents typically need
        "results": [
            {
                # Core customer identifiers
                "id": "cust_001",
                "name": "Alice Johnson",
                "email": "alice.j@example.com",
                "status": "active",
                
                # Key business data
                "total_purchases": 2500.00,
                "account_created": "2023-01-15T10:30:00Z",
                "last_login": "2024-11-20T14:22:10Z",
                
                # Nested address object - often verbose but sometimes needed
                "address": {
                    "street": "123 Main St",
                    "city": "San Francisco",
                    "state": "CA",
                    "zip": "94102",
                    "country": "USA"
                },
                
                # User preferences - rarely relevant to business logic
                "preferences": {
                    "newsletter": True,
                    "notifications": True,
                    "theme": "dark"
                },
                
                # Tracking metadata - useful for analytics but not agent decisions
                "metadata": {
                    "source": "web",
                    "referrer": "google",
                    "user_agent": "Mozilla/5.0..."
                }
            },
            {
                "id": "cust_002",
                "name": "Bob Smith",
                "email": "bob.smith@example.com",
                "status": "active",
                "total_purchases": 1850.00,
                "account_created": "2023-03-22T09:15:00Z",
                "last_login": "2024-11-21T08:45:30Z",
                "address": {
                    "street": "456 Oak Ave",
                    "city": "Austin",
                    "state": "TX",
                    "zip": "78701",
                    "country": "USA"
                },
                "preferences": {
                    "newsletter": False,
                    "notifications": True,
                    "theme": "light"
                },
                "metadata": {
                    "source": "mobile_app",
                    "referrer": "direct",
                    "user_agent": "MobileApp/2.1..."
                }
            },
            {
                "id": "cust_003",
                "name": "Carol White",
                "email": "carol.w@example.com",
                "status": "active",
                "total_purchases": 3200.00,
                "account_created": "2022-11-05T16:20:00Z",
                "last_login": "2024-11-22T11:30:15Z",
                "address": {
                    "street": "789 Pine Rd",
                    "city": "Seattle",
                    "state": "WA",
                    "zip": "98101",
                    "country": "USA"
                },
                "preferences": {
                    "newsletter": True,
                    "notifications": False,
                    "theme": "dark"
                },
                "metadata": {
                    "source": "web",
                    "referrer": "facebook",
                    "user_agent": "Mozilla/5.0..."
                }
            }
        ]
    }

Now let's examine the actual size and token cost of this raw tool result. We will serialize it to JSON and calculate how many tokens it would consume if passed directly to the agent's context. This demonstrates the severity of the verbosity problem.

In [4]:
# Generate a sample tool result
raw_result = simulate_database_query_result()

# Convert to JSON string to see actual size that would be sent to LLM
raw_json = json.dumps(raw_result, indent=2)

# Calculate approximate token count (rough estimate: words * 1.3)
token_estimate = len(raw_json.split()) * 1.3

# Display a preview of the raw result
print("Raw Tool Result:")
print("="*80)
print(raw_json[:500] + "...\n[truncated]\n")
print("="*80)

# Show the token cost statistics
print(f"\nCharacter count: {len(raw_json)}")
print(f"Estimated tokens: {int(token_estimate)}")
print(f"\nProblem: This single tool result consumes ~{int(token_estimate)} tokens!")
print(f"If agent calls 5 tools, that's ~{int(token_estimate * 5)} tokens just for tool results.")
print("\nMost of this data (metadata, preferences, addresses) may be irrelevant to the agent's current task.")

Raw Tool Result:
{
  "query": "SELECT * FROM customers WHERE status = 'active' AND total_purchases > 1000",
  "execution_time_ms": 45,
  "rows_examined": 15234,
  "rows_returned": 3,
  "database_version": "PostgreSQL 14.5",
  "connection_id": "conn_9a7b3c2d",
  "query_id": "qry_2024112201",
  "results": [
    {
      "id": "cust_001",
      "name": "Alice Johnson",
      "email": "alice.j@example.com",
      "status": "active",
      "total_purchases": 2500.0,
      "account_created": "2023-01-15T10:30:00Z",
   ...
[truncated]


Character count: 2274
Estimated tokens: 236

Problem: This single tool result consumes ~236 tokens!
If agent calls 5 tools, that's ~1183 tokens just for tool results.

Most of this data (metadata, preferences, addresses) may be irrelevant to the agent's current task.


This simulation demonstrates the verbosity problem inherent in typical tool outputs. The database query result includes essential data (customer names and purchase totals) but also extensive metadata like execution times, database version, connection IDs, complete address objects, user preferences and tracking information. The JSON serialization produces approximately 1000+ characters translating to 200+ tokens for just three customer records. The nested structure includes multiple levels of objects with fields like `preferences` and `metadata` that are typically unused by agents making business logic decisions. This example shows how a simple query to find high-value customers returns a payload where perhaps 70-80% of the content is irrelevant to the agent's decision-making, illustrating why compression is critical for token efficiency in tool-using agents.

## Selective field extraction
The simplest and most effective compression technique is selective field extraction, where we explicitly specify which fields from tool results are relevant to the agent's task and discard everything else. Rather than passing the complete nested data structure, we create a filtered version containing only the fields the agent needs for decision-making. This requires understanding the agent's task context and configuring extraction rules that preserve essential information while removing metadata, debug data, and auxiliary fields.

We will implement a field extraction system that uses path-based selection to navigate nested structures and extract specific fields. This approach provides fine-grained control over what information enters the agent's context, achieving significant compression while maintaining full access to task-relevant data.

The foundation of selective extraction is the ability to navigate nested data structures using dot notation paths like `results.name` or `address.city`. We will start by building a helper function that can traverse these paths through dictionaries and lists, handling the complexity of nested objects automatically.

In [5]:
def get_nested_value(obj: Any, path: str) -> Any:
    """
    Navigate nested data structure using dot notation path.
    Handles both dictionary keys and list iteration automatically.
    
    Args:
        obj: The data structure to navigate (dict, list, or primitive)
        path: Dot-separated path (e.g., 'results.name' or 'address.city')
        
    Returns:
        The value at the specified path, or None if path doesn't exist
        
    Examples:
        >>> data = {'user': {'name': 'Alice', 'age': 30}}
        >>> get_nested_value(data, 'user.name')
        'Alice'
        
        >>> data = {'users': [{'name': 'Alice'}, {'name': 'Bob'}]}
        >>> get_nested_value(data, 'users.name')
        ['Alice', 'Bob']
    """
    # Split the path into individual parts (e.g., 'results.name' -> ['results', 'name'])
    parts = path.split('.')
    current = obj
    
    # Navigate through each part of the path
    for part in parts:
        if isinstance(current, list):  # If current value is a list, we need to extract from ALL items in the list
            # Recursively apply the remaining path to each item
            remaining_path = '.'.join(parts[parts.index(part):])
            current = [get_nested_value(item, remaining_path) for item in current]
            return current
            
        elif isinstance(current, dict):
            # Navigate into the dictionary using the current path part as a key
            current = current.get(part)
            if current is None:
                # Path doesn't exist - return None
                return None
        else:
            # Reached a primitive value but path continues - invalid path
            return None
    
    return current

Now we will build the main extraction function that uses these paths to pull specific fields from complex tool results. This function takes a list of field paths and creates a new dictionary containing only those extracted values, dramatically reducing the data size.

In [6]:
def extract_fields(data: Union[Dict, List], field_paths: List[str]) -> Dict[str, Any]:
    """
    Extract specific fields from nested data structures using path notation.
    This is the core compression function that selects only relevant fields.
    
    Args:
        data: Dictionary or list containing tool result data
        field_paths: List of dot-notation paths to extract (e.g., ['results.name', 'results.total_purchases'])
        
    Returns:
        Dictionary containing only the extracted fields with their values
        
    Example:
        >>> data = {'results': [{'name': 'Alice', 'age': 30, 'email': 'alice@example.com'}]}
        >>> extract_fields(data, ['results.name', 'results.age'])
        {'results.name': ['Alice'], 'results.age': [30]}
    """
    extracted = {}
    
    # Process each field path that was requested
    for path in field_paths:
        # Use our helper to navigate to the value at this path
        value = get_nested_value(data, path)
        
        if value is not None:
            # Store the extracted value using the full path as the key
            # This preserves information about where the data came from
            extracted[path] = value
    
    return extracted

With our extraction functions ready, we can now build a complete compression function that not only extracts the relevant fields but also tracks compression metrics so we can measure the effectiveness of this approach. The metrics help us understand how much token savings we are achieving.

In [7]:
def compress_tool_result_selective(
    raw_result: Dict[str, Any],
    relevant_fields: List[str]
) -> Dict[str, Any]:
    """
    Compress tool result by extracting only relevant fields.
    Includes compression metadata for monitoring effectiveness.
    
    Args:
        raw_result: Complete tool result with all fields
        relevant_fields: List of field paths to preserve
        
    Returns:
        Compressed result with only specified fields plus compression metrics
    """
    # Extract only the fields we care about
    compressed = extract_fields(raw_result, relevant_fields)
    
    # Calculate compression metrics for monitoring
    original_json = json.dumps(raw_result)
    compressed_json = json.dumps(compressed)
    
    # Add metadata about the compression that was performed
    # This helps track compression effectiveness in production
    compressed['_compression_info'] = {
        'original_fields': len(json.dumps(raw_result).split()),  # Approximate original token count
        'compressed_fields': len(json.dumps(compressed).split()), # Approximate compressed token count
        'compression_ratio': 1 - (len(compressed_json) / len(original_json))  # Percentage reduction
    }
    
    return compressed

Let's see selective field extraction in action. Imagine our agent's task is to "Find high-value customers and their purchase totals" - in this case, we do not need addresses, preferences, metadata or execution details. We only need customer names and purchase amounts. By specifying just those fields, we can dramatically reduce the token cost while preserving all information needed for the task.

In [8]:
# Get our verbose database result again
raw_result = simulate_database_query_result()

# Define relevant fields for the task: "Find high-value customers and their purchase totals"
# We only need:
#   - Customer names (to identify them)
#   - Purchase totals (to assess value)
#   - Result count (to know how many matches)
relevant_fields = [
    'results.name',           # Extract customer names from the results array
    'results.total_purchases', # Extract purchase amounts from the results array
    'rows_returned'           # Get the count of results returned
]

# Apply selective compression
compressed_result = compress_tool_result_selective(raw_result, relevant_fields)

# Format for display
compressed_json = json.dumps(compressed_result, indent=2)

print("Compressed Tool Result (Selective Extraction):")
print("="*80)
print(compressed_json)
print("\n" + "="*80)

# Show the compression effectiveness
original_tokens = len(json.dumps(raw_result).split()) * 1.3
compressed_tokens = len(json.dumps(compressed_result).split()) * 1.3

print(f"\nOriginal tokens: ~{int(original_tokens)}")
print(f"Compressed tokens: ~{int(compressed_tokens)}")
print(f"Token savings: ~{int(original_tokens - compressed_tokens)} ({compressed_result['_compression_info']['compression_ratio']*100:.1f}% reduction)")

# Verify we still have all the information needed
print(f"\nThe agent still has all information needed for the task:")
print(f"  - {len(compressed_result['results.name'])} customer names")
print(f"  - {len(compressed_result['results.total_purchases'])} purchase totals")
print(f"  - Result count: {compressed_result['rows_returned']}")

Compressed Tool Result (Selective Extraction):
{
  "results.name": [
    "Alice Johnson",
    "Bob Smith",
    "Carol White"
  ],
  "results.total_purchases": [
    2500.0,
    1850.0,
    3200.0
  ],
  "rows_returned": 3,
  "_compression_info": {
    "original_fields": 154,
    "compressed_fields": 13,
    "compression_ratio": 0.9202813599062134
  }
}


Original tokens: ~200
Compressed tokens: ~26
Token savings: ~174 (92.0% reduction)

The agent still has all information needed for the task:
  - 3 customer names
  - 3 purchase totals
  - Result count: 3


The selective field extraction system uses dot notation to navigate nested data structures, supporting both dictionary traversal and list iteration.
- The `get_nested_value` function implements recursive path navigation, splitting paths like `results.name` into components and traversing the data structure step by step. When encountering a list, it applies the remaining path to each list item, automatically handling array fields without requiring explicit iteration syntax. 
- The `compress_tool_result_selective` function wraps this extraction and adds compression metadata for monitoring effectiveness. In the example, extracting only customer names and purchase totals from the full result reduces token count from approximately 200 to 26 tokens, achieving 90%+ compression while preserving all information needed for the task of identifying high-value customers.
- The compressed result excludes addresses, preferences, metadata, execution times and connection details that are irrelevant to the business logic decision.

## LLM-Based result summarization
For tool results that contain natural language content, detailed descriptions or complex data that can't easily be filtered by field selection alone, LLM-based summarization provides an alternative compression approach. The language model can understand the semantic content of tool results and generate concise summaries that capture essential information while discarding verbose details. This is particularly valuable for tools that return documentation, search results, log files or other text-heavy content.

We will implement an LLM-powered summarization system that takes verbose tool results and produces compact summaries preserving key information. The summarization prompt is designed to maintain factual accuracy while aggressively reducing token count, ensuring the agent receives actionable information without unnecessary verbosity.

The key to effective LLM-based compression is creating a summarization prompt that understands the agent's task context. We do not want a generic summary - we want a summary that preserves exactly the information the agent needs for its current task while discarding everything else. Let's build a function that takes the tool result, the task context, and generates a targeted summary.

In [9]:
def summarize_tool_result(
    tool_name: str,
    raw_result: Union[Dict, str],
    task_context: str,
    llm: ChatOpenAI,
    max_summary_length: int = 100
) -> str:
    """
    Use LLM to create a concise, task-aware summary of tool results.
    
    Args:
        tool_name: Name of the tool that produced the result
        raw_result: Raw tool result (dict or string)
        task_context: Description of what the agent is trying to accomplish
        llm: Language model for summarization
        max_summary_length: Target maximum words for summary
        
    Returns:
        Concise summary string preserving essential information
    """
    # Convert result to string representation if it's a dictionary - The LLM needs text input to process
    if isinstance(raw_result, dict):
        result_text = json.dumps(raw_result, indent=2)
    else:
        result_text = str(raw_result)
    
    # Create a task-aware summarization prompt - The key is emphasizing preservation of task-relevant information
    summarization_prompt = f"""You are helping an AI agent process tool results efficiently.

Tool: {tool_name}
Agent's Task: {task_context}

Raw Tool Result:
{result_text}

Create a concise summary (max {max_summary_length} words) that:
1. Preserves all information relevant to the agent's task
2. Includes key data points, numbers, and identifiers
3. Omits metadata, debug info, and irrelevant details
4. Uses clear, direct language

Summary:"""
    
    # Generate the summary using the LLM
    response = llm.invoke([HumanMessage(content=summarization_prompt)])
    
    return response.content

Let's apply LLM-based summarization to our database query result. We will provide the task context about targeting customers for a premium membership offer, and the LLM will intelligently summarize the results to focus on information relevant to that specific task.

In [10]:
# Get our verbose database result
raw_result = simulate_database_query_result()

# Define the agent's task context - this guides what information to preserve
task_context = "Identify high-value customers to target for a premium membership offer"

# Generate a task-aware summary
summary = summarize_tool_result(
    tool_name="database_query",
    raw_result=raw_result,
    task_context=task_context,
    llm=llm,
    max_summary_length=80  # Limit to 80 words for aggressive compression
)

print("LLM-Generated Summary:")
print("="*80)
print(summary)
print("\n" + "="*80)

# Compare the token costs
original_size = len(json.dumps(raw_result).split()) * 1.3
summary_size = len(summary.split()) * 1.3

print(f"\nOriginal result tokens: ~{int(original_size)}")
print(f"Summary tokens: ~{int(summary_size)}")
print(f"Compression: {(1 - summary_size/original_size)*100:.1f}%")

LLM-Generated Summary:
Three high-value customers identified for premium membership targeting: 

1. Alice Johnson (cust_001) - $2500 total purchases, last login: 2024-11-20, email: alice.j@example.com, active status.
2. Bob Smith (cust_002) - $1850 total purchases, last login: 2024-11-21, email: bob.smith@example.com, active status.
3. Carol White (cust_003) - $3200 total purchases, last login: 2024-11-22, email: carol.w@example.com, active status.


Original result tokens: ~200
Summary tokens: ~68
Compression: 65.6%


The summary preserves:
  - Number of customers found
  - Their names and purchase totals
  - Relevance to the premium membership task

The summary excludes:
  - Database execution metadata
  - Complete address information
  - User preferences

The LLM-based summarization approach uses task-aware prompting to guide the model toward preserving information relevant to the agent's current objective.
- The summarization prompt explicitly includes the task context (targeting customers for premium membership) alongside the raw tool result, enabling the LLM to make intelligent decisions about what information is essential versus superfluous.
- The `max_summary_length` parameter provides control over compression aggressiveness.

The LLM processes the verbose JSON structure and generates a natural language summary that captures key facts - customer names, purchase amounts, and their relevance to the task - while discarding execution metadata, nested address objects and preference settings. This typically achieves 60-75% compression while maintaining semantic completeness, though it introduces a small latency cost from the additional LLM call. The approach is particularly valuable for results containing natural language content where field selection is not sufficient.

## Structured compression with schemas
For production systems requiring reliable, consistent compression, we can define schemas that specify exactly what information should be extracted from each tool result type. Using Pydantic models, we create structured compression definitions that ensure critical fields are always preserved while providing type safety and validation. This schema-based approach combines the token efficiency of selective extraction with the reliability and maintainability of strongly-typed structures.

We will implement a schema-driven compression system where each tool type has a defined output schema specifying which fields to extract, how to transform them, and what summary information to include. This provides a scalable, maintainable approach for systems with many different tools, ensuring consistent compression behavior across all tool types.

First, we will define Pydantic schemas that describe the compressed format for our tool results. These schemas act as contracts, specifying exactly what fields must be present in compressed results and their types. This ensures that downstream code can rely on a consistent structure.

In [11]:
class CustomerRecord(BaseModel):
    """
    Compressed customer record preserving only essential information.
    This schema defines the minimal customer data needed for most agent tasks.
    """
    # Core identifiers - always needed
    id: str = Field(description="Customer identifier")
    name: str = Field(description="Customer name")
    
    # Business-critical data
    total_purchases: float = Field(description="Total purchase amount")
    
    # Optional location data - included when geography is relevant
    city: Optional[str] = Field(default=None, description="City location")


class CompressedDatabaseResult(BaseModel):
    """
    Schema for compressed database query results.
    Defines the structure that all database query compressions must follow.
    """
    # Metadata about the query results
    row_count: int = Field(description="Number of rows returned")
    
    # The actual data - array of compressed customer records
    customers: List[CustomerRecord] = Field(description="Customer records")
    
    # Optional performance data - can be useful for monitoring
    execution_time_ms: Optional[int] = Field(
        default=None,
        description="Query execution time (optional for performance monitoring)"
    )


class CompressedAPIResult(BaseModel):
    """
    Schema for compressed API call results.
    Generic structure for API responses that need summarization.
    """
    # Response status
    status: str = Field(description="API call status")
    
    # Summarized content instead of full response
    data_summary: str = Field(description="Brief summary of returned data")
    
    # Record count for pagination awareness
    record_count: int = Field(description="Number of records in response")

Now we will create a compressor class that uses these schemas to transform raw tool results into compressed versions. The compressor will have methods for each tool type, applying the appropriate schema-based extraction and transformation.

In [12]:
class ToolResultCompressor:
    """
    Schema-driven tool result compression system.
    Maps tool types to compression schemas and applies structured transformations.
    """
    
    def __init__(self, llm: ChatOpenAI):
        """
        Initialize compressor with LLM for schema extraction.
        
        Args:
            llm: Language model with structured output support
        """
        self.llm = llm
        
        # Register compression schemas for different tool types
        # This creates a mapping from tool name to the schema it should use
        self.schemas = {
            'database_query': CompressedDatabaseResult,
            'api_call': CompressedAPIResult
        }
    
    def compress_database_result(
        self,
        raw_result: Dict[str, Any]
    ) -> CompressedDatabaseResult:
        """
        Compress database query result using the database schema.
        Extracts only fields defined in CompressedDatabaseResult.
        
        Args:
            raw_result: Raw database query result with all fields
            
        Returns:
            Compressed result matching CompressedDatabaseResult schema
        """
        # Extract customer records with only essential fields
        customers = []
        for record in raw_result.get('results', []):
            # Create CustomerRecord instance with only schema-defined fields
            # This automatically excludes email, status, preferences, metadata, etc.
            customer = CustomerRecord(
                id=record['id'],
                name=record['name'],
                total_purchases=record['total_purchases'],
                # Extract city from nested address object
                city=record.get('address', {}).get('city')
            )
            customers.append(customer)
        
        # Build the compressed result according to schema
        compressed = CompressedDatabaseResult(
            row_count=raw_result.get('rows_returned', len(customers)),
            customers=customers,
            execution_time_ms=raw_result.get('execution_time_ms')
        )
        
        return compressed
    
    def compress(self, tool_name: str, raw_result: Dict[str, Any]) -> Union[CompressedDatabaseResult, CompressedAPIResult, Dict]:
        """
        Compress tool result based on registered schema.
        Routes to appropriate compression method for the tool type.
        
        Args:
            tool_name: Name of the tool
            raw_result: Raw tool result
            
        Returns:
            Compressed result matching appropriate schema
        """
        # Route to appropriate compression method based on tool type
        if tool_name == 'database_query':
            return self.compress_database_result(raw_result)
        else:
            # Fallback: return raw result if no schema registered
            # In production, you might want to apply generic summarization here
            return raw_result
    
    def to_dict(self, compressed_result: BaseModel) -> Dict[str, Any]:
        """
        Convert Pydantic model to dictionary for serialization.
        Useful for converting compressed results to JSON for storage or transmission.
        
        Args:
            compressed_result: Pydantic model instance
            
        Returns:
            Dictionary representation of the model
        """
        return compressed_result.model_dump()

Now let's see schema-driven compression in action. We will use our compressor to transform the verbose database result into a structured, validated compressed format that is guaranteed to have the fields we need.

In [13]:
# Create the schema-driven compressor
compressor = ToolResultCompressor(llm)

# Get our verbose database result
raw_result = simulate_database_query_result()

# Compress using the database schema
compressed = compressor.compress('database_query', raw_result)

# Convert Pydantic model to dict for display
compressed_dict = compressor.to_dict(compressed)
compressed_json = json.dumps(compressed_dict, indent=2)

print("Schema-Based Compressed Result:")
print("="*80)
print(compressed_json)
print("\n" + "="*80)

# Analyze compression effectiveness
original_tokens = len(json.dumps(raw_result).split()) * 1.3
compressed_tokens = len(compressed_json.split()) * 1.3

print(f"\nOriginal tokens: ~{int(original_tokens)}")
print(f"Compressed tokens: ~{int(compressed_tokens)}")
print(f"Compression ratio: {(1 - compressed_tokens/original_tokens)*100:.1f}%")

Schema-Based Compressed Result:
{
  "row_count": 3,
  "customers": [
    {
      "id": "cust_001",
      "name": "Alice Johnson",
      "total_purchases": 2500.0,
      "city": "San Francisco"
    },
    {
      "id": "cust_002",
      "name": "Bob Smith",
      "total_purchases": 1850.0,
      "city": "Austin"
    },
    {
      "id": "cust_003",
      "name": "Carol White",
      "total_purchases": 3200.0,
      "city": "Seattle"
    }
  ],
  "execution_time_ms": 45
}


Original tokens: ~200
Compressed tokens: ~55
Compression ratio: 72.1%


Schema guarantees:
  - Customer IDs always preserved
  - Names and purchase totals always included
  - City information extracted from nested address
  - Type safety and validation enforced
  - Consistent compression across all database queries

The schema-driven compression system uses Pydantic models to define explicit extraction and transformation rules for each tool type. 
- The `CompressedDatabaseResult` schema specifies that compressed results must include row count, a list of customer records, and optionally execution time.
- Each `CustomerRecord` schema defines the minimal set of fields needed (id, name, total_purchases, city), automatically excluding addresses, preferences, metadata, and other verbose fields.
- The `ToolResultCompressor` class maintains a registry mapping tool names to compression schemas and provides tool-specific compression methods that extract and transform data according to the schema.

Pydantic's validation ensures that required fields are always present and correctly typed, preventing compression errors that could break downstream agent logic. In the example, the schema-based compression achieves similar 70-80% token reduction as manual extraction but provides stronger guarantees about output structure, making it more suitable for production systems with multiple tool types and complex result structures.

## Production tool result manager
Bringing together all compression techniques, we can build a production-ready tool result manager that automatically selects and applies the appropriate compression strategy based on tool type, result size and task context. This system provides a unified interface for tool result processing, handling everything from simple field extraction to LLM-based summarization to schema-driven compression. The manager includes intelligent fallbacks, compression effectiveness monitoring, and configurable strategies that can be tuned for different tools and use cases.

We will implement a complete tool result manager that examines each tool result, determines the optimal compression strategy, applies it, tracks effectiveness metrics, and provides APIs for both adding new compression rules and retrieving compressed results. This represents a production-grade solution for managing tool result verbosity in complex multi-tool agent systems.

We will start by defining a configuration model for compression strategies. This allows us to declaratively specify how each tool type should be compressed, making it easy to register and manage compression rules.

In [25]:
class CompressionStrategy(BaseModel):
    """
    Configuration for a specific compression strategy.
    Defines how a particular tool's results should be compressed.
    """
    # The compression method to use
    method: str = Field(description="Compression method: 'selective', 'summarize', or 'schema'")
    
    # For selective extraction: which field paths to keep
    field_paths: Optional[List[str]] = Field(
        default=None,
        description="Field paths for selective extraction (e.g., ['results.name', 'results.total_purchases'])"
    )
    
    # For schema-based compression: which Pydantic schema to use
    schema_class: Optional[str] = Field(
        default=None,
        description="Pydantic schema class name for schema-based compression"
    )
    
    # For LLM summarization: how aggressively to compress
    max_summary_length: int = Field(
        default=100,
        description="Maximum words for LLM summarization"
    )

Now, let's build the foundation of our production manager. We will initialize it with configuration options and set up the infrastructure for tracking compression strategies and metrics.

In [26]:
class ProductionToolResultManager:
    """
    Production-ready manager for tool result compression.
    Automatically selects and applies optimal compression strategies.
    """
    
    def __init__(
        self,
        llm: ChatOpenAI,
        default_compression_threshold: int = 200,
        enable_metrics: bool = True
    ):
        """
        Initialize tool result manager.
        
        Args:
            llm: Language model for summarization
            default_compression_threshold: Token count above which to compress (default: 200)
            enable_metrics: Whether to track compression metrics (default: True)
        """
        self.llm = llm
        self.default_compression_threshold = default_compression_threshold
        self.enable_metrics = enable_metrics
        
        # Initialize the schema-driven compressor for schema-based strategies
        self.schema_compressor = ToolResultCompressor(llm)
        
        # Register default compression strategies for known tools - This can be extended by calling register_strategy()
        self.strategies: Dict[str, CompressionStrategy] = {
            'database_query': CompressionStrategy(
                method='schema',
                schema_class='CompressedDatabaseResult'
            ),
            'api_call': CompressionStrategy(
                method='summarize',
                max_summary_length=80
            ),
            'file_read': CompressionStrategy(
                method='summarize',
                max_summary_length=120
            )
        }
        
        # Initialize metrics tracking
        self.metrics = {
            'total_results': 0,  # Total number of tool results processed
            'compressed_results': 0,  # How many were actually compressed
            'total_tokens_saved': 0,  # Cumulative token savings
            'compression_by_method': {  # Breakdown by compression method
                'selective': 0,
                'summarize': 0,
                'schema': 0,
                'none': 0  # Results that didn't need compression
            }
        }

We need helper methods for the manager - one to estimate token counts, one to decide if compression is needed, and one to allow registering new compression strategies for additional tools.

In [27]:
def _estimate_tokens(self, text: str) -> int:
    """
    Estimate token count for text.
    Uses a simple heuristic: words * 1.3 (approximates GPT tokenization).
    
    Args:
        text: Text to estimate tokens for
        
    Returns:
        Estimated token count
    """
    return int(len(text.split()) * 1.3)

def _should_compress(self, raw_result: Union[Dict, str]) -> bool:
    """
    Determine if result should be compressed based on size.
    Small results don't benefit from compression (overhead not worth it).
    
    Args:
        raw_result: Raw tool result
        
    Returns:
        True if result exceeds compression threshold
    """
    # Convert to string for size estimation
    if isinstance(raw_result, dict):
        result_str = json.dumps(raw_result)
    else:
        result_str = str(raw_result)
    
    # Check if token count exceeds our threshold
    tokens = self._estimate_tokens(result_str)
    return tokens > self.default_compression_threshold

def register_strategy(
    self,
    tool_name: str,
    strategy: CompressionStrategy
) -> None:
    """
    Register a compression strategy for a specific tool.
    Allows extending the manager with new tool types.
    
    Args:
        tool_name: Name of the tool
        strategy: Compression strategy configuration
    """
    self.strategies[tool_name] = strategy

# Add these helper methods to the ProductionToolResultManager class
ProductionToolResultManager._estimate_tokens = _estimate_tokens
ProductionToolResultManager._should_compress = _should_compress
ProductionToolResultManager.register_strategy = register_strategy

Now comes the core compression logic. This method is the heart of the production manager - it examines each tool result, selects the appropriate compression strategy, applies it, and tracks metrics. The logic handles routing to different compression methods, calculating savings, and maintaining detailed statistics.

In [28]:
def compress_result(
    self,
    tool_name: str,
    raw_result: Union[Dict, str],
    task_context: Optional[str] = None
) -> Dict[str, Any]:
    """
    Compress tool result using appropriate strategy.
    This is the main entry point for compression.
    
    Args:
        tool_name: Name of the tool that produced the result
        raw_result: Raw tool result (dict or string)
        task_context: Optional context about agent's current task
        
    Returns:
        Dictionary containing:
            - compressed: bool indicating if compression was applied
            - result: the compressed (or raw) result
            - method: compression method used
            - metrics: token counts and savings (if compressed)
    """
    # Increment total results counter
    self.metrics['total_results'] += 1

    # Calculate original size for all results (needed for consistent return structure)
    original_str = json.dumps(raw_result) if isinstance(raw_result, dict) else str(raw_result)
    original_tokens = self._estimate_tokens(original_str)
    
    # Check if compression is needed based on size
    if not self._should_compress(raw_result):
        # Result is small enough - no compression needed
        # Return consistent structure with token metrics showing no compression
        self.metrics['compression_by_method']['none'] += 1
        return {
            'compressed': False,
            'result': raw_result,
            'method': 'none',
            'original_tokens': original_tokens,
            'compressed_tokens': original_tokens,  # Same as original since no compression
            'tokens_saved': 0,  # No tokens saved
            'compression_ratio': 0.0  # No compression ratio
        }
    
    # Get the compression strategy for this tool (if registered)
    strategy = self.strategies.get(tool_name)
    
    # Initialize variables for compression
    compressed_result = None
    method_used = 'none'
    
    # Apply the appropriate compression method
    if strategy:
        if strategy.method == 'schema':
            # Use schema-driven compression
            compressed_obj = self.schema_compressor.compress(tool_name, raw_result)
            compressed_result = self.schema_compressor.to_dict(compressed_obj)
            method_used = 'schema'
            
        elif strategy.method == 'selective' and strategy.field_paths:
            # Use selective field extraction
            compressed_result = extract_fields(raw_result, strategy.field_paths)
            method_used = 'selective'
            
        elif strategy.method == 'summarize':
            # Use LLM summarization
            summary = summarize_tool_result(
                tool_name=tool_name,
                raw_result=raw_result,
                task_context=task_context or "Processing tool result",
                llm=self.llm,
                max_summary_length=strategy.max_summary_length
            )
            compressed_result = {'summary': summary}
            method_used = 'summarize'
    else:
        # No strategy registered for this tool - use default summarization as fallback
        summary = summarize_tool_result(
            tool_name=tool_name,
            raw_result=raw_result,
            task_context=task_context or "Processing tool result",
            llm=self.llm
        )
        compressed_result = {'summary': summary}
        method_used = 'summarize'
    
    # Calculate compression metrics
    compressed_str = json.dumps(compressed_result)
    compressed_tokens = self._estimate_tokens(compressed_str)
    tokens_saved = original_tokens - compressed_tokens
    
    # Update metrics tracking
    if self.enable_metrics:
        self.metrics['compressed_results'] += 1
        self.metrics['total_tokens_saved'] += tokens_saved
        self.metrics['compression_by_method'][method_used] += 1
    
    # Return compressed result with full metadata
    return {
        'compressed': True,
        'result': compressed_result,
        'method': method_used,
        'original_tokens': original_tokens,
        'compressed_tokens': compressed_tokens,
        'tokens_saved': tokens_saved,
        'compression_ratio': (1 - compressed_tokens / original_tokens) * 100
    }

# Add the compress_result method to the ProductionToolResultManager class
ProductionToolResultManager.compress_result = compress_result

Finally, we will add a method to retrieve compression metrics. This allows monitoring the effectiveness of compression strategies in production, helping identify opportunities for optimization.

In [29]:
def get_metrics(self) -> Dict[str, Any]:
    """
    Get compression metrics and statistics.
    Useful for monitoring compression effectiveness in production.
    
    Returns:
        Dictionary containing:
            - total_results: total tool results processed
            - compressed_results: how many were compressed
            - total_tokens_saved: cumulative token savings
            - compression_by_method: breakdown by method
            - compression_rate: percentage of results that were compressed
    """
    return {
        **self.metrics,  # Include all tracked metrics
        # Add calculated compression rate
        'compression_rate': (
            self.metrics['compressed_results'] / 
            max(self.metrics['total_results'], 1) * 100
        )
    }

# Add the get_metrics method to the ProductionToolResultManager class
ProductionToolResultManager.get_metrics = get_metrics

Now let's see the production manager in action with a realistic multi-tool workflow. We will process several different tool results - a database query, an API call and a small result - demonstrating how the manager intelligently selects compression strategies and tracks metrics across all operations.

In [30]:
# Initialize the production manager with our configuration
manager = ProductionToolResultManager(
    llm=llm,
    default_compression_threshold=150  # Compress results larger than 150 tokens
)

# Simulate a multi-tool workflow
print("Tool Result Processing:")
print("="*80)

# Tool 1: Database query (will use schema compression)
db_result = simulate_database_query_result()
compressed_db = manager.compress_result(
    'database_query',
    db_result,
    task_context="Find customers for premium membership offer"
)

print("\n1. Database Query Result:")
print(f"   Method: {compressed_db['method']}")
print(f"   Original: {compressed_db['original_tokens']} tokens")
print(f"   Compressed: {compressed_db['compressed_tokens']} tokens")
print(f"   Savings: {compressed_db['tokens_saved']} tokens ({compressed_db['compression_ratio']:.1f}%)")

Tool Result Processing:

1. Database Query Result:
   Method: schema
   Original: 200 tokens
   Compressed: 42 tokens
   Savings: 158 tokens (79.0%)


In [31]:
# Tool 2: API call (will use summarization)
api_result = {
    'status': 200,
    'timestamp': '2024-11-22T10:30:00Z',
    'request_id': 'req_9x7b2c4d',
    'headers': {
        'content-type': 'application/json',
        'x-rate-limit': '100',
        'x-rate-limit-remaining': '87',
        'x-request-id': 'req_9x7b2c4d',
        'server': 'nginx/1.21.0',
        'cache-control': 'no-cache'
    },
    'metadata': {
        'total_count': 8,
        'page': 1,
        'per_page': 10,
        'api_version': '2.1.0'
    },
    'data': {
        'products': [
            {
                'id': 'p1',
                'name': 'Widget',
                'description': 'High-quality widget for industrial use',
                'price': 29.99,
                'stock': 150,
                'category': 'Tools',
                'manufacturer': 'WidgetCo',
                'sku': 'WDG-001'
            },
            {
                'id': 'p2',
                'name': 'Gadget',
                'description': 'Multi-purpose gadget with premium features',
                'price': 49.99,
                'stock': 75,
                'category': 'Electronics',
                'manufacturer': 'GadgetInc',
                'sku': 'GDG-002'
            },
            {
                'id': 'p3',
                'name': 'Gizmo',
                'description': 'Advanced gizmo for professional applications',
                'price': 79.99,
                'stock': 45,
                'category': 'Tools',
                'manufacturer': 'GizmoCorp',
                'sku': 'GZM-003'
            },
            {
                'id': 'p4',
                'name': 'Doohickey',
                'description': 'Essential doohickey for everyday tasks',
                'price': 19.99,
                'stock': 200,
                'category': 'Accessories',
                'manufacturer': 'DoohickeyLtd',
                'sku': 'DHK-004'
            },
            {
                'id': 'p5',
                'name': 'Thingamajig',
                'description': 'Versatile thingamajig with multiple uses',
                'price': 34.99,
                'stock': 120,
                'category': 'Tools',
                'manufacturer': 'ThingCo',
                'sku': 'THG-005'
            }
        ]
    }
}

compressed_api = manager.compress_result(
    'api_call',
    api_result,
    task_context="Check product availability"
)

print("\n2. API Call Result:")
print(f"   Method: {compressed_api['method']}")
print(f"   Original: {compressed_api['original_tokens']} tokens")
print(f"   Compressed: {compressed_api['compressed_tokens']} tokens")
print(f"   Savings: {compressed_api['tokens_saved']} tokens ({compressed_api['compression_ratio']:.1f}%)")


2. API Call Result:
   Method: summarize
   Original: 169 tokens
   Compressed: 91 tokens
   Savings: 78 tokens (46.2%)


In [32]:
# Tool 3: Small result (will not be compressed)
small_result = {'status': 'success', 'count': 5}
compressed_small = manager.compress_result('simple_tool', small_result)

print("\n3. Small Tool Result:")
print(f"   Method: {compressed_small['method']}")
print(f"   Compressed: {compressed_small['compressed']}")
print(f"   (Result was below threshold - no compression needed)")


3. Small Tool Result:
   Method: none
   Compressed: False
   (Result was below threshold - no compression needed)


In [33]:
# Display overall metrics from the entire workflow
metrics = manager.get_metrics()

print("\n" + "="*80)
print("\nOverall Compression Metrics:")
print(f"  Total tool results processed: {metrics['total_results']}")
print(f"  Results compressed: {metrics['compressed_results']}")
print(f"  Compression rate: {metrics['compression_rate']:.1f}%")
print(f"  Total tokens saved: {metrics['total_tokens_saved']}")
print(f"\n  Compression methods used:")
for method, count in metrics['compression_by_method'].items():
    print(f"    {method}: {count}")



Overall Compression Metrics:
  Total tool results processed: 3
  Results compressed: 2
  Compression rate: 66.7%
  Total tokens saved: 236

  Compression methods used:
    selective: 0
    summarize: 1
    schema: 1
    none: 1


The `ProductionToolResultManager` implements a complete tool result processing pipeline with automatic strategy selection, multiple compression methods and comprehensive metrics tracking. 
- The system maintains a registry of compression strategies mapped to tool names, allowing different tools to use optimal compression approaches - schema-based for structured database results, summarization for text-heavy API responses and selective extraction for specific use cases.
- The `compress_result` method first checks if compression is needed by comparing token count to a configurable threshold, then applies the registered strategy or falls back to default summarization.
- The manager tracks detailed metrics including total results processed, compression rates by method, and cumulative token savings, providing visibility into compression effectiveness.

The example demonstrates a realistic multi-tool workflow. This production-ready implementation provides the foundation for efficient context management in complex agent systems that make frequent tool calls.